In [177]:
#import necessary libaries
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [178]:
#load the csv file and drop the weight column
csv_file = "athlete_events.csv"
olypmic_df = pd.read_csv(csv_file)
olypmic_df = olypmic_df.drop(['Weight'], axis=1)

In [179]:
#fill na types in height data type with 0's to be filterable
olypmic_df.Height=olypmic_df.Height.fillna(value=0)

In [180]:
#capture unique names of countries and their NOC abbreviation
countires = olypmic_df.Team.unique()
countryNoc =  olypmic_df.NOC.unique()
#fill na types with no medal in 'medals' column to be filterable
olypmic_df.Medal=olypmic_df.Medal.fillna(value="no_medal")

In [181]:
#creates 3 lists that hold the numbers of medals each country has earned
country_gold_list = list()
country_silver_list = list()
country_bronze_list = list()

for val in countryNoc:
    newdf = olypmic_df.loc[olypmic_df.NOC==val]
    country_gold_list.append(len(newdf.loc[newdf.Medal=="Gold"]))
for val in countryNoc:
    newdf = olypmic_df.loc[olypmic_df.NOC==val]
    country_silver_list.append(len(newdf.loc[newdf.Medal=="Silver"]))
for val in countryNoc:
    newdf = olypmic_df.loc[olypmic_df.NOC==val]
    country_bronze_list.append(len(newdf.loc[newdf.Medal=="Bronze"]))

230 230


In [182]:
#creates a dataframe that holds each country gold, silver, bronze, and total counts
country_medal_df = pd.DataFrame(countryNoc)
country_medal_df["Gold_Medals"]=country_gold_list
country_medal_df["Silver_Medals"]=country_silver_list
country_medal_df["Bronze_Medals"]=country_bronze_list
#changes '0' column name into 'NOC'
country_medal_df=country_medal_df.rename(columns={0:"NOC"})
#sets the sum of each countries total medal count into the dataframe
country_medal_df["Total_Medals"]=country_medal_df.sum(axis=1)
country_medal_df.head()

,NOC,Gold_Medals,Silver_Medals,Bronze_Medals,Total_Medals
0,CHN,350,347,292,989
1,DEN,179,241,177,597
2,NED,287,340,413,1040
3,USA,2638,1641,1358,5637
4,FIN,198,270,432,900


In [183]:
#filters out data of for height values of 0
heights_df = olypmic_df[olypmic_df.Height>0]
#sorts height data be sex
heights_m_df =  heights_df[heights_df.Sex=="M"]
heights_f_df =  heights_df[heights_df.Sex=="F"]

In [184]:
#groups eight values by year
m_avg_h=heights_m_df.groupby("Year")["Height"].mean()
f_avg_h=heights_f_df.groupby("Year")["Height"].mean()
#creates two sepereate height data frames by sex
mheight_df =  pd.DataFrame(m_avg_h)
fheight_df =  pd.DataFrame(f_avg_h)

In [185]:
#merges both data frames
mheight_df['Female_Height']=fheight_df.Height
combined_height=mheight_df.rename(columns={"Height":"Male_Height"})

In [186]:
#renames columns
combined_height=combined_height.rename(columns={"Year":"year","Male_Height":"male_height","Female_Height":"female_height"})

In [187]:
#creating and setting primary key
x=len(combined_height.male_height)
val=[]
t=0
while t < x:
    val.append(t+1)
    t+=1
val
combined_height["id"]=val
combined_height["year"]=combined_height.index
combined_height=combined_height.set_index('id')
country_medal_df.head()

,NOC,Gold_Medals,Silver_Medals,Bronze_Medals,Total_Medals
0,CHN,350,347,292,989
1,DEN,179,241,177,597
2,NED,287,340,413,1040
3,USA,2638,1641,1358,5637
4,FIN,198,270,432,900


In [188]:
#creating and setting primary keys
val=[]
x = len(country_medal_df.NOC)
val=[]
t=0
while t<x:
    val.append(t+1)
    t+=1
country_medal_df['id']=val
country_medal_df=country_medal_df.set_index('id')
country_medal_df.head()

,NOC,Gold_Medals,Silver_Medals,Bronze_Medals,Total_Medals
id,,,,,
1,CHN,350,347,292,989
2,DEN,179,241,177,597
3,NED,287,340,413,1040
4,USA,2638,1641,1358,5637
5,FIN,198,270,432,900


In [189]:
#saving data frames as csvs
olypmic_df.to_csv("olympic_data.csv")
country_medal_df.to_csv("country_medal.csv")
combined_height.to_csv("combined_height_avg.csv")

In [190]:
#creating a connection to pgadmin database
rds_connection_string = "postgres:postgres@localhost:5432/olympic_stats"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [192]:
#creates and fills in tables
combined_height.to_sql(name='combined_height_average', con=engine, if_exists='append', index=True)

In [193]:
country_medal_df.to_sql(name='country_medal_stats', con=engine, if_exists='append', index=True)

In [1]:
#pd.read_sql_query('select * from country_medal_stats', con=engine).head()

In [195]:
import csv_to_sqlite

In [196]:
#creating sqlite files for falsk
options=csv_to_sqlite.CsvOptions(typing_style="full",encoding="windows-1250")
csv_to_sqlite.write_csv(["country_medal.csv"], "medals.sqlite", options)
csv_to_sqlite.write_csv(["combined_height_avg.csv"], "height.sqlite", options)
#csv_to_sqlite.write_csv(["olympic_data.csv"], "olympics.sqlite", options)

Written 230 rows into 1 tables in 0.024 seconds
Written 35 rows into 1 tables in 0.018 seconds


35